In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import lib.data_loaders as data_loaders
import lib.data_transformers as data_transformers
import lib.models_repo as models_repo
import lib.optimizer_repo as optimizer_repo
import lib.scheduler_repo as scheduler_repo
import lib.trainer as trainer

In [3]:
from torchvision.datasets.folder import ImageFolder

In [4]:
%matplotlib inline

In [5]:
data_path = "/home/as/datasets/fastai.dogscats"
num_classes = 2
img_size  = 224
batch_size = 128

In [6]:
norm = data_transformers.resnet_normaliser
trans = data_transformers.get_transformer(img_size, norm, False)
trans_aug = data_transformers.get_transformer(img_size, norm, True)
trans_valid = data_transformers.get_test_valid_transformer(img_size, norm)

In [7]:
# Vanilla dataset
train_images = ImageFolder(f'{data_path}/train', transform=trans)
# Vanilla dataset
train_images_aug = ImageFolder(f'{data_path}/train', transform=trans_aug)
# For valid and test datasets
valid_images = ImageFolder(f'{data_path}/valid', transform=trans_valid)

In [8]:
print('Number of train instances', len(train_images))
print('Number of valid instances', len(valid_images))

Number of train instances 23000
Number of valid instances 2000


In [9]:
print('Classes', train_images.classes)
print('Class index', train_images.class_to_idx)

Classes ['cats', 'dogs']
Class index {'cats': 0, 'dogs': 1}


In [10]:
train_loader = data_loaders.get_data_loader(train_images, batch_size)
train_loader_aug = data_loaders.get_data_loader(train_images_aug, batch_size)
valid_loader = data_loaders.get_data_loader(valid_images, batch_size)

In [11]:
if False:
    model = models_repo.resnet_vanilla(num_classes)
    criteria, optimizer = optimizer_repo.sgd(model, 0.01)
    scheduler = scheduler_repo.step_lr(optimizer)

    best_model = trainer.experiment(model, criteria, optimizer, scheduler, train_loader, valid_loader, 10)

In [12]:
if False:
    model = models_repo.resnet_vanilla(num_classes)
    criteria, optimizer = optimizer_repo.sgd(model, 0.01)
    scheduler = scheduler_repo.step_lr(optimizer)

    best_model = trainer.experiment(model, criteria, optimizer, scheduler, train_loader_aug, valid_loader, 10)

In [18]:
model, params_to_optimize = models_repo.resnet_extra_layers(num_classes, top_layers_to_freeze=7, debug=True)

<class 'torch.nn.modules.conv.Conv2d'> 1 0
<class 'torch.nn.modules.batchnorm.BatchNorm2d'> 2 0
<class 'torch.nn.modules.activation.ReLU'> 0 0
<class 'torch.nn.modules.pooling.MaxPool2d'> 0 0
<class 'torch.nn.modules.container.Sequential'> 18 0
<class 'torch.nn.modules.container.Sequential'> 27 0
<class 'torch.nn.modules.container.Sequential'> 39 0
<class 'torch.nn.modules.container.Sequential'> 21 21
<class 'lib.models_repo.AdaptiveConcatPool2d'> 0 0
<class 'lib.models_repo.Flatten'> 0 0
<class 'torch.nn.modules.batchnorm.BatchNorm1d'> 2 2
<class 'torch.nn.modules.dropout.Dropout'> 0 0
<class 'torch.nn.modules.linear.Linear'> 2 2
<class 'torch.nn.modules.activation.ReLU'> 0 0
<class 'torch.nn.modules.batchnorm.BatchNorm1d'> 2 2
<class 'torch.nn.modules.dropout.Dropout'> 0 0
<class 'torch.nn.modules.linear.Linear'> 2 2
<class 'torch.nn.modules.activation.LogSoftmax'> 0 0


In [19]:
model, params_to_optimize = models_repo.resnet_extra_layers(num_classes, top_layers_to_freeze=7)
criteria, optimizer = optimizer_repo.sgd(model, 0.01, params_to_optimize=params_to_optimize)
scheduler = scheduler_repo.step_lr(optimizer)

best_model = trainer.experiment(model, criteria, optimizer, scheduler, train_loader_aug, valid_loader, 10)

Total training instances: 23000
Total validation instances: 2000
Classes: ['cats', 'dogs']


A Jupyter Widget


Epoch 1/10


180it [00:33,  5.37it/s]
16it [00:02,  5.93it/s]

Training Loss  : 0.00048050932067891826 , Acc: 0.9771304347826087
Validation Loss: 0.00019258361333049834 , Acc: 0.9905
############################### Better model found

Epoch 2/10


...

Training Loss  : 8.683344847573291e-05 , Acc: 0.9965652173913043
Validation Loss: 0.0001437762964051217 , Acc: 0.992

Epoch 9/10



180it [00:38,  4.68it/s]
16it [00:02,  5.56it/s]

Training Loss  : 8.747086450255112e-05 , Acc: 0.9963913043478261
Validation Loss: 0.00014347369794268162 , Acc: 0.9925

Epoch 10/10



180it [00:37,  4.75it/s]
16it [00:02,  5.54it/s]

Training Loss  : 7.754367307769945e-05 , Acc: 0.9971739130434782
Validation Loss: 0.0001496546813286841 , Acc: 0.9915

